# 3 - Pré-processamento e transformações

Nessa fase, construiremos alguns modelos específicos para texto para então treiná-los;

In [1]:
# caminho para instalação do pacote mltoolkit, com metricas e gráficos personalizados
# !pip install git+ssh://git@github.com/flimao/mltoolkit

In [2]:
import pandas as pd
import numpy as np
from matplotlib import rcParams, rcParamsDefault, pyplot as plt
import seaborn as sns
from mltoolkit import metrics, plots, NLP
import spacy

rcParams.update(rcParamsDefault)
rcParams['figure.dpi'] = 120
rcParams['figure.figsize'] = (10, 8)

nlp = spacy.load("pt_core_news_lg")

## Importação dos dados

Primeiramente, importamos os dados e aplicamos as transformações utilizadas na fase anterior:

In [3]:
# não tocaremos no conjunto de submissão

tweets_raw = pd.read_csv(
    r'../data/Train3Classes.csv',
)

In [4]:
# trocar tipos para acelerar o processamento (menos espaço em memória)
# e ativar possíveis otimizações internas ao pandas para certos tipos
def mudar_tipos(df):
    df = df.copy()

    df['id'] = df['id'].astype('string')
    df['tweet_date'] = pd.to_datetime(df['tweet_date'])
    df['sentiment'] = df['sentiment'].astype('category')

    return df

def remover_duplicatas(df):
    df = df.copy()

    df = df.drop_duplicates(subset = 'id')

    return df

# o índice é o id, visto que não há repetidos
# vantagem: o índice é removido automaticamente quando separamos em base de treino e teste.
def setar_index(df):
    df = df.copy()

    df = df.set_index('id')

    return df

tweets_full = (tweets_raw
    .pipe(mudar_tipos)
    .pipe(remover_duplicatas)
    .pipe(setar_index)
)

## Pré-processamento de texto

Vamos então implementar o pré-processamento do texto da fase anterior (Análise Exploratória de Texto).

Primeiramente vamos importar as *stopwords*:

In [11]:
with open(r'../data/stopwords_alopes.txt', encoding = 'utf8') as stopword_list:
    lst = stopword_list.read().splitlines()

stopwords_alopes = set([ stopword.strip() for stopword in lst ])

# em uma análise de sentimento, não queremos remover palavras com conotação negativa
remover_stopwords = {
    'não', 
}

stopwords_alopes -= remover_stopwords

In [12]:
preprocessing_full = lambda s: NLP.preprocessing(s, preproc_funs_args = [
    NLP.remove_links,
    NLP.remove_hashtags,
    NLP.remove_mentions,
    NLP.remove_numbers,
    NLP.remove_special_caract,
    NLP.lowercase,
    #remove_punkt,
    #(remove_stopwords, dict(stopword_list = stopword_list_alopes)),
    (NLP.tokenize_remove_stopwords_get_radicais_spacy, dict(
        nlp = nlp,
        stopword_list = stopwords_alopes,
    )),
])

Vamos então aplicar esse pré-processamento a uma amostra da base de *tweets* (para podermos iterar rapidamente caso necessário). 

Em um momento posterior, treinaremos a base completa.

In [9]:
amostra_eda = 5000
radicais = tweets_full.sample(amostra_eda)['tweet_text'].apply(preprocessing_full)

tweets = tweets_full.copy()
tweets['radicais'] = radicais
tweets = tweets[tweets.radicais.notna()]

In [13]:
tweets.sample(10)

,tweet_text,tweet_date,sentiment,query_used,radicais
id,,,,,
1034257540736065537,Olha eu preparando um novo projeto gráfico. Pa...,2018-08-28 01:53:07+00:00,2,#trabalho,olhar preparar projeto grafico pausar pra foti...
1050742090152468480,@lfrancobastos Porque não atendeste? Fifa ? :P,2018-10-12 13:36:50+00:00,1,:),nao atender fifa :p
1046254666147860481,Fizemos o grupinho das ARMYs de cwb e ele já f...,2018-09-30 04:25:25+00:00,0,:(,fazer grupo armys cwb ja flopou querer convers...
1030566551357927424,Assistente Operacional (Emissão de NF) - Serra...,2018-08-17 21:26:27+00:00,2,#oportunidade,assistente operacional emissao nf serrar es
1049058097610854400,Ibope: Boca de urna dá segundo turno entre Bol...,2018-10-07 22:05:15+00:00,2,jornaloglobo,ibope bocar urna turno bolsonaro haddad presid...
1031482217548144642,BRASIL/EMPREGOS Construa seu futuro conosco: o...,2018-08-20 10:04:59+00:00,2,#trabalho,brasil emprego construir futurar conosco ofert...
1045392580949741568,@RIC4RDO_2479 @anais_lachado18 Btw tu não sabe...,2018-09-27 19:19:48+00:00,1,:),btw nao saber gostar
1050752935603384320,@juzaraujo Que daora! Fico contente que tenha ...,2018-10-12 14:19:56+00:00,1,:),daora ficar contentar dar certar querer ha tri...
1047492196419686401,@Erickaolol O básico é ganhar dos caras ir lá ...,2018-10-03 14:22:55+00:00,1,:),basico ganhar caro o melhor ta mundo mal kabum...


## Opções de modelos

Vamos agora olhar para alguns modelos que podemos utilizar.

Definiremos os modelos desejados, e então procederemos à comparação dos mesmos.

In [17]:
X = tweets['radicais']
y = tweets['sentiment']

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size = 0.3,
    stratify = y,
)

### 1. *Bag of Words* / `CountVectorizer`

*Bag of Words* é o processo onde traduzimos o texto já tratado para uma representação numérica que faça sentido para o modelo de *Machine Learning* consiga interpretá-lo.

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
X_train_cv = cv.fit_transform(X_train).toarray()
X_test_cv = cv.transform(X_test).toarray()

In [25]:
X_train_cv

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [31]:
X_train_cv.shape

(3500, 6191)

### 2. TF-IDF

***Term Frequency and Inverse Document Frequency*** é uma tranformação onde avaliamos a relevância das palavras pela **Frequência dos Termos** e multiplicamos pelo **Inverso da Frequência nos Documentos**.

Nesse contexto, um **documento** é cada um dos textos dentro de um *dataset*. Vamos entender cada um dos termos:

> **TF - Term Frequency**: é a frequência de vezes que um termo/palavra aparece em cada um dos documentos analisados (isso nos ajuda a avaliar a relevância daquela palavra);

> **IDF - Inverse Document Frequency**: aqui avaliamos em quantos documentos o termo/palavra aparece (dessa forma conseguimos entender a sua influência em identificar os textos);

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(use_idf = True)

X_train_tfidf = tfidf.fit_transform(X_train).todense()
X_test_tfidf  = tfidf.transform(X_test).todense()

In [28]:
X_train_tfidf

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
X_train_tfidf.shape

(3500, 6191)

### *Word2vec*

O *Word2vec* é um modelo onde associa-se vetores a cada palavra. Os vetores são tais que pretendem capturar as relações semânticas entre as mesmas.

Por exemplo, se tivermos em nosso vocabulário as palavras *rei*, *rainha*, *homem* e *mulher*, poderíamos fazer a seguinte operação vetorial:

$ \vec{v}_{rei} - \vec{v}_{homem} + \vec{v}_{mulher} = \vec{v}_{rainha}$

In [64]:
from gensim.models import Word2Vec

X_train_tokens = X_train.str.split(' ').to_list()
X_test_tokens = X_test.str.split(' ').to_list()

w2v_model = Word2Vec(
    sentences = X_train_tokens, 
    vector_size = 2,  # este parâmetro é o equivalente ao número de features. 
    min_count = 1, 
    workers = 2
)

In [55]:
w2v_model.wv.most_similar(positive = ['bolsonaro'], negative = ['haddad'])

[('thrones', 1.0),
 ('rafaela', 1.0),
 ('anem', 0.9999998807907104),
 ('vendido', 0.9999968409538269),
 ('hyunjinni', 0.9999948740005493),
 ('matematicamente', 0.9999947547912598),
 ('leriar', 0.9999938011169434),
 ('adnet', 0.9999933838844299),
 ('eisec', 0.9999922513961792),
 ('us$', 0.9999921917915344)]

In [56]:
w2v_model.wv.similarity('bolsonaro', 'haddad')

0.77390736

### *Doc2Vec*

O *Doc2Vec* é um modelo similar ao *Word2Vec*, mas que leva em consideração também o contexto de cada frase na construção dos vetores de similaridade.

In [65]:
from gensim.models import doc2vec

def read_corpus(list_sentences, tokens_only=False):
    if tokens_only:
        return list_sentences
    else:
        # For training data, add tags
        lista = []
        for i, line in enumerate(list_sentences):
            lista.append(doc2vec.TaggedDocument(line, [i]))

        return lista
    
train_corpus = read_corpus(X_train_tokens)
test_corpus = read_corpus(X_test_tokens, tokens_only=True)

d2v_model = doc2vec.Doc2Vec(vector_size = 50, min_count = 2, epochs = 20)

d2v_model.build_vocab(train_corpus)

d2v_model.train(
    train_corpus, 
    total_examples = d2v_model.corpus_count, 
    epochs = d2v_model.epochs)

In [60]:
d2v_model.infer_vector(['bolsonaro', 'haddad'])

array([-0.03627553,  0.01049397,  0.02647647,  0.01406603, -0.01712412,
       -0.05749064,  0.00441333,  0.07438195, -0.09029654, -0.02020809,
       -0.00110922, -0.03923719, -0.0037813 ,  0.02054167, -0.02334482,
       -0.00499761,  0.02373652,  0.0089486 , -0.04906051, -0.03406813,
        0.01519929,  0.04696488,  0.03257749, -0.02575215,  0.04330669,
        0.02003675, -0.01625567, -0.01424281, -0.04072778,  0.01125126,
        0.00036247, -0.01422183, -0.00379539,  0.03486306, -0.0267536 ,
        0.03035104, -0.00334509, -0.00667038,  0.02646112, -0.04137891,
        0.05067763,  0.00484286, -0.02319106, -0.00397156,  0.06552546,
       -0.00272405,  0.01524034, -0.05292337,  0.02484859,  0.01479054],
      dtype=float32)

In [66]:
# medir o desempenho
X_train_d2v = []

for phrase in X_train_tokens:
    vecs = []
    vecs.append(d2v_model.infer_vector(phrase))
    
    X_train_d2v.append(vecs)
    
X_train_d2v = np.array(X_train_d2v)[:, 0, :]

In [67]:
# medir o desempenho
X_test_d2v = []

for phrase in X_test_tokens:
    vecs = []
    vecs.append(d2v_model.infer_vector(phrase))
    
    X_test_d2v.append(vecs)
    
X_test_d2v = np.array(X_test_d2v)[:, 0, :]

In [68]:
X_train_d2v.shape

(3500, 50)

In [70]:
X_test_d2v.shape

(1500, 50)